In [ ]:
%matplotlib inline



# Gaze mapping pipeline

This example shows how to perform post-hoc pupil detection, calibration marker
detection, calibration and gaze mapping.

<div class="alert alert-info"><h4>Note</h4><p>This example requires the dependencies for
    `pupil detection<pupil_detection_dependencies>` as well as
    ``matplotlib``.</p></div>



In [ ]:
import pupil_recording_interface as pri

## Set recording folder



In [ ]:
folder = pri.get_test_recording()

## Create readers for video streams



In [ ]:
world_reader = pri.VideoReader(folder)
eye0_reader = pri.VideoReader(folder, stream="eye0", color_format="gray")
eye1_reader = pri.VideoReader(folder, stream="eye1", color_format="gray")

## Detect pupils



In [ ]:
pupil_detector_eye0 = pri.PupilDetector(camera_id=0)
pupil_list_eye0 = pupil_detector_eye0.batch_run(eye0_reader)

pupil_detector_eye1 = pri.PupilDetector(camera_id=1)
pupil_list_eye1 = pupil_detector_eye1.batch_run(eye1_reader)

## Detect calibration markers



In [ ]:
marker_detector = pri.CircleDetector()
marker_list = marker_detector.batch_run(world_reader)

## Merge pupil data and run calibration



In [ ]:
calibration = pri.Calibration(resolution=world_reader.resolution)
pupil_list = pri.merge_pupils(pupil_list_eye0, pupil_list_eye1)
calibration_result = calibration.batch_run(pupil_list, marker_list)

## Map gaze as dataset



In [ ]:
gaze_mapper = pri.GazeMapper(calibration=calibration_result)
gaze = gaze_mapper.batch_run(
    pupil_list, return_type="dataset", info=world_reader.info
)

## Plot gaze



In [ ]:
norm_pos = gaze.gaze_norm_pos.where(gaze.gaze_confidence_2d > 0.7)
norm_pos.plot.line(x="time")